# Dérivation

In [ ]:
%reset -f

In [ ]:
import tensorflow as tf
import torch
import jax
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("default")

## Introduction

Mais comment calcule-t-on des dérivées ? Pas sur du papier, mais avec un ordinateur ? La bonne réponse est venue tardivement (1985 environ) et a permis le développement du deep-learning.  


### Dérivation composée (Chain rule)

Détaillons les maths sur un exemple

Analysons la dérivée de la fonction $  h \circ g \circ f (x)  $.  Voici son graphe de calcul:

$$
x \xrightarrow  f   y   \xrightarrow g z   \xrightarrow h  t
$$
  Les accroissements infinitésimaux se multiplient (c'est la chain rule) :

\begin{alignat}{1}
\frac {\partial  t  }{\partial x}      &=        \frac{ \partial y }{ \partial x}   \frac{ \partial z }{ \partial y}   \frac{ \partial t }{ \partial z } \\
&=  f'(x) g'(y) h'(z)    
\end{alignat}






## Passage forward puis dérivation backward

Nous allons décortiquer le calcul de cette dérivée en un point précis. Pour fixer les idées:

* $f(a) = \sin(a)$, donc $f'(a) = \cos(a)$
* $g(a) =  4 a^2$, donc $g'(a) = 8 a$
* $h(a)=  \tanh(a)$ donc $h'(a)= 1-\tanh^2(a)$

Notons que l'ordinateur sait évaluer précisément ces fonctions élémentaire. Et de plus, les lib comme Jax,tensorflow, torch savent associer à une fonction élémentaire, sa dérivéee (ce n'est pas le cas de numpy).

 Maintenant il s'agit de bien composer les calculs. Nous voulons calculer
$$
   \frac{\partial (h \circ g \circ f )(x)} {\partial x}  
$$
en $x=7$.

Forward pass:
1. Calcul et stockage de $y=f(x)$
2. Calcul et stockage de $z=g(y)$
3. Calcul et stockage de $t=h(z)$

Backward pass:
1. Calcul de $\frac{\partial t}{\partial z} = h'(z)$
2. Calcul de $\frac{\partial t}{\partial y} = g'(y) h'(z)$.
3. Calcul de $\frac{\partial t}{\partial x} = f'(x) g'(y) h'(z)$.



### Implémentations

In [ ]:
x=tf.Variable(7.)
with tf.GradientTape(persistent=False) as tape:
    t = tf.tanh(4*tf.cos(x)**2)

print(tape.gradient(t,x).numpy())

In [ ]:
x=torch.tensor(7.,requires_grad=True)
t = torch.tanh(4*torch.cos(x)**2)
t.backward()
print(x.grad)

In [ ]:
x=jax.numpy.array(7.)
fn = lambda x: jax.numpy.tanh(4*jax.numpy.cos(x)**2)
grad = jax.grad(fn)(x)
print(grad)

***À vous:***  Considérons des scalaires $a,b,c,d$ et les fonctions affines

* $f(x) = ax+b $ et
* $g(x) = cx + d $.


Calculez explicitement $g\circ f(x+\epsilon) - g\circ f(x)$.    


En comparant cet exo et la 'chain rule', vous comprendrez que : les accroissements infinitésimaux des fonctions lisses, se composent de la même manière que les accroissements des fonctions affines.  En bref : toute fonction lisse est localement une fonction affine.


### Régle de l'accumulation

Quand une fonction a plusieurs variables, $g(a,b,...)$, ses dérivées partielles  se calculent sans difficulté. Par ex, pour calculer  $\frac{\partial g(a,b,...)}{\partial a}$ il suffit de considérer uniquement la fonction $a\to g(a,b,...)$.


Par contre, quand  une variable $x$ intervient plusieurs fois:
$$
z=h(x) =  g  [ f_1 (x), f_2 (x) , ...] = g [ y_1,y_2,...]
$$
 Graphe de calcul (dit en diamant):
$$
x \xrightarrow f  \begin{bmatrix}  y_1 \\  y_2 \\ \vdots \end{bmatrix}  \xrightarrow g z
 $$
 Les accroissements s'additionnent (s'accumulent):
$$
\frac {\partial  z  }{\partial x}  =    \sum_i        \frac{\partial y_i }{\partial x}      \frac {\partial z }{\partial y_i} =    \sum_i     f'_i(x) g'(y_i)  
$$


***À vous:*** vous connaissez par cœur la régle de dérivation d'un produit:
$$
(f_1 * f_2)' = f'_1 f_2 + f_1 f'_2
$$
Vérifiez qu'il s'agit d'un cas particulier de la régle d'accumulation. Pour vous aider, considérer le graphe de calcul en diamant:
$$
x \xrightarrow f  \begin{bmatrix}  f_1(x) \\  f_2(x)  \end{bmatrix}  \xrightarrow * f_1(x) * f_2(x)
 $$







Vérifions la régle de l'accumulation en tensorflow:

In [ ]:
x=tf.Variable(7.)

with tf.GradientTape(persistent=True) as tape:
    y1=x**2
    y2=tf.cos(x)
    y3=tf.atan(x)
    z=y1*y2/y3

print(tape.gradient(z,x).numpy())

dz_dy1=tape.gradient(z,y1)
dz_dy2=tape.gradient(z,y2)
dz_dy3=tape.gradient(z,y3)

dy1_dx=tape.gradient(y1,x)
dy2_dx=tape.gradient(y2,x)
dy3_dx=tape.gradient(y3,x)

dz_dx = dz_dy1*dy1_dx + dz_dy2*dy2_dx + dz_dy3*dy3_dx
print(dz_dx.numpy())

### Comparaison avec le calcul formel "symbolique"

In [ ]:
def fonction_complexe(x,y,z):
    a=atan(x/y)
    b=cos(z**2-x)
    return x*y*a*b/z+a-b

In [ ]:
%%time
from tensorflow import cos,atan

x=tf.Variable(7.)
y=tf.Variable(5.)
z=tf.Variable(2.)
with tf.GradientTape() as tape:
    f=fonction_complexe(x,y,z)

[df_dx,df_dy,df_dz]=tape.gradient(f,[x,y,z])
print(df_dx.numpy())
print(df_dy.numpy())
print(df_dz.numpy())

In [ ]:
%%time
import sympy
from sympy import cos,atan

x,y,z=sympy.symbols('x y z')
f=fonction_complexe(x,y,z)
df_dx=sympy.Derivative(f, x).doit()
df_dy=sympy.Derivative(f, y).doit()
df_dz=sympy.Derivative(f, z).doit()

subs={x:7.,y:5.,z:2.}
print(df_dx.evalf(subs=subs))
print(df_dy.evalf(subs=subs))
print(df_dz.evalf(subs=subs))

Regardons les expressions que doit retenir `sympy`

In [ ]:
print(df_dx)

### graph de calul

C'est le détails des calculs

Voici un graphe de calcul de `z=(x*a)**2`. Il faut le lire de bas en haut.

    x     a
     \   /
       y=x*a
       |
       z=y**2

Remarquons qu'il n'y a pas de cycle dans un graph de calcul, sinon on ne serez pas faire le calcul. Par contre il peut y avoir des diaments, ce qui oblige à utiliser la régle d'accumulation.

### Second exemple

Suivons l'exemple du calcul de dérivée de
$$
z=(x^2*cos(x))^2
$$
Le graph des calcul inclus un diamant puisque $x$ intervient deux fois.

Forward



        x=𝜋
       /   \
    a=x^2   b=cos(x)
     =𝜋²    =-1
     \      /
       y=a*b
        =-𝜋²
        |
       z=y**2
        =𝜋⁴

Backward



Etape 1


    dz/dy=2y
         =-2𝜋²


            





Etape 2
       

      dz/da           dz/db
      =dz/dy*dy/da    =dz/dy*dy/db
      =-2𝜋² *b        =-2𝜋² *a
      =2𝜋²            =-2𝜋⁴
        \            /  
           dz/dy=-2𝜋²

Etape 3

            dz/dx
            =  dz/da*da/dx
              +dz/db*db/dx
            =  2𝜋²* 2x
              -2𝜋⁴* (-sin(x))
            = 4𝜋³
          /          \

      dz/da           dz/db
      =2𝜋²            =-2𝜋⁴
        \            /  
           dz/dy=-2𝜋²

### Calculer l'occupation de la mémoire gpu

Redémarez la session.

In [ ]:
import torch

In [ ]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.max_memory_allocated()

Vérifiez que vous êtes bien à 0 ci-dessus. Sinon cela signifie que vous avez avant construits des tenseurs dans le gpu.

In [ ]:
size=1024

In [ ]:
A=torch.ones(size,device="cuda")
torch.cuda.max_memory_allocated()

In [ ]:
size*4

In [ ]:
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.max_memory_allocated()

In [ ]:
A=torch.ones(size,dtype=torch.float64,device="cuda")
torch.cuda.max_memory_allocated()

In [ ]:
size*8

In [ ]:
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.max_memory_allocated()

***À vous:*** Que se passe-t-il si l'on remplace 1024 par une taille légèrement plus grande, ou plus petite ? Vous en déduirez pourquoi on aime bien définir des tenseurs dont les tailles sont des puissances de 2.

***À vous:*** Que vérifie-t-on dans la suite ?

In [ ]:
def some_calculus(requires_grad,n):
    A=torch.rand(1000,device="cuda",requires_grad=requires_grad)
    for _ in range(n):
        A=A*torch.rand(1000,device="cuda")
    return A

In [ ]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.max_memory_allocated()

In [ ]:
torch.cuda.reset_peak_memory_stats()
A=some_calculus(True,10)
print(torch.cuda.max_memory_allocated())
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
A=some_calculus(True,100)
print(torch.cuda.max_memory_allocated())
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
A=some_calculus(False,10)
print(torch.cuda.max_memory_allocated())
del A

In [ ]:
torch.cuda.reset_peak_memory_stats()
A=some_calculus(False,100)
print(torch.cuda.max_memory_allocated())
del A

### Jax champion de la simplicité

Dans les années passées j'ai beaucoup pratiqué torch et tensorflow, et il y a beaucoup de chose bizarre quand on dérive.

Jax est plus simple est souvent plus performant

Et en plus, il est proche des maths: car on dérive des fonctions! et pas des résultats d'évaluations de fonctions.

In [ ]:
%reset -f

In [ ]:
import jax.numpy as jnp
import jax
from jax import grad, jit, vmap
import matplotlib.pyplot as plt

In [ ]:
f_of_x = lambda x:x**2

f_of_x_dx = grad(f_of_x)

f_of_xV = vmap(f_of_x)
f_of_xV_dx = vmap(f_of_x_dx)

In [ ]:
xV = jnp.linspace(-5,5,100)
fig,ax=plt.subplots(figsize=(10,6))
ax.plot(xV,f_of_xV(xV),label="f(x)")
ax.plot(xV,f_of_xV_dx(xV),label="f'(x)")
ax.legend();

C'est normal que l'on ne puisse pas faire:

    grad(f_of_x)(xV)

Dans la logique Jax, on travaille avec des fonctions à valeur scalaire, et on les vectorisent juste avant leur évaluation.

### Jitons

In [ ]:
import jax.numpy as jnp
import jax
from jax import grad, jit, vmap
import jax.random as jr
import matplotlib.pyplot as plt

In [ ]:
%%time
def f_of_Θ_x(Θ,x):
    return jnp.tanh(Θ@jnp.tanh(Θ@x))

def loss(Θ,x,y):
    return jnp.mean((f_of_Θ_x(Θ,x)-y)**2)

def loss_grad(Θ,x,y):
    return grad(loss)(Θ,x,y)


Θ = jr.normal(jr.PRNGKey(0), (100, 100))
x = jr.normal(jr.PRNGKey(1), (100,))
y = jr.normal(jr.PRNGKey(2), ())

dΘ = loss_grad(Θ,x,y)
dΘ.shape

In [ ]:
def compute_loss_and_dΘ():
    Θ = jr.normal(jr.PRNGKey(0), (100, 100))
    x = jr.normal(jr.PRNGKey(1), (100,))
    y = jr.normal(jr.PRNGKey(2), ())

    dΘ = loss_grad(Θ,x,y)
    print(dΘ.shape)

    return loss(Θ,x,y),dΘ

In [ ]:
%%time
_ = compute_loss_and_dΘ()

In [ ]:
compute_loss_and_dΘ_jit=jit(compute_loss_and_dΘ)
_ = compute_loss_and_dΘ_jit()

In [ ]:
%%time
_ = compute_loss_and_dΘ_jit()

On peut encore légèrement améliorer les performances en utilisant `jax.value_and_grad` ce qui économise un passage forward.  

In [ ]:
def loss_value_and_grad(Θ,x,y):
    return jax.value_and_grad(loss)(Θ,x,y)